In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime

<h1>Limpieza y filtrado de la data.<h1>

In [2]:
name = r"C:\Users\jibar\OneDrive - Universidad de los andes\Clases_Maestria\CIENCIA DE DATOS\Proyecto\General\General\Data\Daily.csv"

data =pd.read_csv(name,sep='|',error_bad_lines=False, warn_bad_lines= False,index_col=False,header=0, low_memory=False)

In [3]:
def eliminar_coma(x):
    x=str(x).replace(',','.')
    return x

In [4]:

data['VolUnCorrected']=data['VolUnCorrected'].apply(eliminar_coma)
data['VolCorrected']=data['VolCorrected'].apply(eliminar_coma)
data['Pressure']=data['Pressure'].apply(eliminar_coma)
data['Temperature']=data['Temperature'].apply(eliminar_coma)


convert = {'id_sitio':'object',
          'VolUnCorrected':float,
          'VolCorrected':float,
          'Pressure':float,
          'Temperature':float,
          'MaxPressure':float,
          'MinPressure':float,
          'MaxFlow':float,
          'MinFlow':float,
          'MaxTemp':float,
          'MinTemp':float,
          'id_RTU':'object'}
data = data.astype(convert)

In [5]:
selected = ['id_sitio','Daily_Date','VolUnCorrected',
       'VolCorrected', 'Pressure', 'Temperature']
data  = data[selected]

In [6]:
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

In [7]:
data['Daily_Date']   = pd.to_datetime(data['Daily_Date'])
data['Day'] =data['Daily_Date'].dt.date
lista = list(range(151,170))
data1 = data[data['id_sitio'].isin(lista)]

lista = [153, 154, 157, 160, 165, 167]
data1 = data[data['id_sitio'].isin(lista)]
grupo1 = data1.copy()

lista = [151, 152, 155, 156, 158, 159, 161, 162, 163, 164, 166, 168, 169, 170]
data1 = data[data['id_sitio'].isin(lista)]
grupo2 = data1.copy()

In [19]:
def etiquetado(data=grupo2 ,id_sitio = 151, cons =3, std=2, vent =30):
        mapa = grupo2['id_sitio']==id_sitio # que sitio 
        consecutivos = cons # condicion para valores repetidos seguidos infdicador de falla
        stds =std #Condicion de deviaciones estandart para identificador de falla (picos)
        ventana =vent # Ventana de prediccion



        inst = data[mapa].copy()
        day = inst['Day']
        daily =inst['Daily_Date'] 
        ids = inst['id_sitio'].copy()
        inst.drop(['id_sitio','Day'],axis = 1,inplace = True)
        scaler = StandardScaler()
        stan = scaler.fit_transform(inst.drop(['Daily_Date'], axis = 1))

        datframe = pd.DataFrame(data=stan,columns=inst.drop(['Daily_Date'],axis = 1).columns)
        datframe['Daily_Date'] = daily.reset_index(drop=True)
        datframe['id_sitio'] = ids.reset_index(drop=True)
        
        datframe.sort_values(by='Daily_Date',inplace = True)
        datframe.reset_index(inplace=True,drop=True)


        datframe['Mayor_2std'] = (datframe['VolUnCorrected']>stds) | (datframe['VolUnCorrected']<-stds)



        lista =[]


        for i in range(0,datframe.index.max()+1):
            if i<3:
                adelante = (datframe['VolUnCorrected'].iloc[i:i+consecutivos].mean()==datframe['VolUnCorrected'].iloc[i].mean())
                lista.append(adelante)
            elif i>datframe.index.max()-3:
                atras = (datframe['VolUnCorrected'].iloc[i-consecutivos:i].mean()==datframe['VolUnCorrected'].iloc[i].mean())
                lista.append(atras)
            else :
                atras = (datframe['VolUnCorrected'].iloc[i-consecutivos:i].mean()==datframe['VolUnCorrected'].iloc[i].mean())
                adelante = (datframe['VolUnCorrected'].iloc[i:i+consecutivos].mean()==datframe['VolUnCorrected'].iloc[i].mean())
                consecutivo= atras | adelante
                lista.append(consecutivo)
        datframe['Consecutivo']=lista

        datframe['Falla']=(datframe['Mayor_2std'] & datframe['Consecutivo'])


        lista = []

        for i in range(0,datframe.index.max()+1):
            if datframe.index.max()-(ventana-1)>i:
                lista.append(datframe['Falla'].iloc[i:i+ventana].any())
            else :
                lista.append(False)
        datframe['Predictor']=lista

        datframe['SoloAntes']=(datframe['Falla'] ^ datframe['Predictor'])

        datframe['FallaNum'] = datframe['Falla'].map({True:3,False:0})
        datframe['PredictorNum'] = datframe['Predictor'].map({True:2,False:0})
        datframe['SoloAntesNum'] = datframe['SoloAntes'].map({True:1,False:0})
        return datframe[['VolUnCorrected', 'VolCorrected', 'Pressure', 'Temperature','Daily_Date', 'id_sitio',  'SoloAntesNum']]

In [41]:
ids = grupo2['id_sitio'].unique().tolist()
data_etiquetada = pd.DataFrame()
if (155 in ids) and (168 in ids):
    ids.remove(155)
    ids.remove(168)
for i in ids:
    ventana = 30
    sal = etiquetado( data=grupo2 ,id_sitio = i, cons =3, std=2, vent =ventana)
    
  
    ## aquie genercion de carcateristicas.
    
    

    

    new_characteristics = pd.DataFrame()
    
    for i in range(ventana,sal.index.max()+1):
        
        caracteriticas =['media', 'mediana','std']

#         timestep.drop(['Daily_Date','id_sitio'],axis =1,inplace =True)
        timestep = sal.iloc[i-ventana:i].drop(['Daily_Date','id_sitio'],axis =1)

        media = timestep.mean()

        mediana = timestep.median()

        desviacion = timestep.std()


        Dmediana = {i:j for i, j in zip(mediana.index,('Mediana_'+mediana.index))}

        Dmedia = {i:j for i, j in zip(mediana.index,('Media_'+media.index))}

        Ddesviacion = {i:j for i, j in zip(desviacion.index,('Desviacion_'+desviacion.index))}

        mediana.rename(Dmediana, inplace = True)
        media.rename(Dmedia, inplace = True)  
        desviacion.rename(Ddesviacion , inplace = True) 

        med = pd.DataFrame(media).transpose()
        mian=pd.DataFrame(mediana).transpose()
        desv = pd.DataFrame(desviacion).transpose()
        actual = (pd.DataFrame(sal.iloc[i]).transpose()).reset_index()


    #     new_characteristics = new_characteristics.append(pd.concat([med,mian,desv,actual],axis = 1))
        # Hasta aca funciona sin lo siguiente.

        primeras = pd.concat([med,mian,desv,actual],axis = 1)
        for i in range(29,0,-1):
            newVol = '{} Dias_antesVol'.format(i)
            newTemp = '{} Dias_antesTemp'.format(i)
            newVPres = '{} Dias_antesPres'.format(i)
            names_list =[]
            value_list=[]
            names_list.append(newVol)
            names_list.append(newTemp)
            names_list.append(newVPres)

    #         names_list=[newVol,newTemp,newVPres]

            value_list.append([timestep.iloc[30-i]['VolUnCorrected']])
            value_list.append([timestep.iloc[30-i]['Temperature']])
            value_list.append([timestep.iloc[30-i]['Pressure']])
    #         value_list=[timestep.iloc[30-i]['VolUnCorrected'],timestep.iloc[30-i]['Temperature'],timestep.iloc[30-i]['Pressure']]


            segundas =pd.DataFrame(data=value_list,index=names_list)

            primeras = pd.concat([primeras,segundas.transpose()],axis = 1)
    #         new_characteristics[newVol]=ventana[30-i]['VolUnCorrected']
    #         new_characteristics[newTemp]=ventana[30-i]['Temperature']
    #         dias[newPres]=ventana[30-i]['Pressure']
#         new_characteristics = new_characteristics.append(pd.concat([actual,med,mian,desv,primeras],axis = 1))    
        new_characteristics = new_characteristics.append(primeras)    

    
    
    
    data_etiquetada=data_etiquetada.append(new_characteristics, ignore_index = True)


In [27]:
data_etiquetada.shape

(18887, 113)

In [42]:
data_etiquetada.to_csv(r'datos_etiquetados_con_caracteristicas2V.csv')